In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
file_id = '1K3I8BPQY-CCFZKC2gEyuQBZMW_GjAEFC'
link = 'https://drive.google.com/uc?id=1K3I8BPQY-CCFZKC2gEyuQBZMW_GjAEFC'
data = pd.read_csv(link)
print(data.head())

In [ ]:
!pip install attention

In [ ]:
import attention

In [ ]:
from tensorflow.keras.layers import Attention

In [ ]:
pip install --upgrade keras

In [ ]:
pip install --upgrade tensorflow

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
from keras.preprocessing.text import Tokenizer
from keras.utils.data_utils import pad_sequences
# from keras.io.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
from tensorflow.keras.layers import Input, Embedding, Dense, Concatenate, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import warnings
pd.set_option("display.max_colwidth", 200)
warnings.filterwarnings("ignore")


In [ ]:
import pandas as pd

In [ ]:
data=pd.read_csv("https://drive.google.com/uc?id=1K3I8BPQY-CCFZKC2gEyuQBZMW_GjAEFC", sep=',', encoding='utf-8')

print(data.head())

In [ ]:
data.drop_duplicates(subset=['Text'], inplace=True)
data.dropna(axis=0,inplace=True)

In [ ]:
data.Info()

Preprocessing

In [ ]:
contraction_mapping = {
    "ain't": "is not",
    "aren't": "are not",
    "can't": "cannot",
    "cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "he'll": "he will",
    "he's": "he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how is",
    "I'd": "I would",
    "I'd've": "I would have",
    "I'll": "I will",
    "I'll've": "I will have",
    "I'm": "I am",
    "I've": "I have",
    "i'd": "i would",
    "i'd've": "i would have",
    "i'll": "i will",
    "i'll've": "i will have",
    "i'm": "i am",
    "i've": "i have",
    "isn't": "is not",
    "it'd": "it would",
    "it'd've": "it would have",
    "it'll": "it will",
    "it'll've": "it will have",
    "it's": "it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she would",
    "she'd've": "she would have",
    "she'll": "she will",
    "she'll've": "she will have",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so as",
    # Add more key-value pairs here if needed
}


In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
stop_words = set(stopwords.words('english'))
def text_cleaner (text,num):
    newString = text.lower()
    newString = BeautifulSoup (newString, "1xml").text
    newString =re.sub(r'\([^)]*\)','',newString)
    newString = re.sub('"',"", newString)
    newString = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in newString.split(" ")])
    newString = re.sub(r"'s\b","",newString)
    newString = re.sub("[a-zA-Z]", "", newString)
    newString = re.sub('[m]{2,)', 'mm', newString)
    if (num==0):
      tokens = [w for w in newString.split() if not w in stop_words]
    else:
      tokens= newString.split()
    long_words=[]
    for i in tokens:
        if len(i)>1:
            long_words.append(1)
    return (" ".join(long_words)).strip()


In [ ]:
cleaned_text = []
for t in data['Text']:
    cleaned_text.append(text_cleaner(t,0))

In [ ]:
cleaned_text[:5]

[]

In [ ]:
cleaned_summary = []
for t in data['Summary']:
    cleaned_summary.append(text_cleaner(t,1))

In [ ]:
cleaned_summary[:10]

[]

In [ ]:
data['cleaned_text']=cleaned_text
data['cleaned_summary']=cleaned_summary

In [ ]:
data.head(1)

In [ ]:
data.replace('', np.nan, inplace=True)
data.dropna(axis=0,inplace=True)

In [ ]:
import matplotlib.pyplot as plt
text_word_count=[]
summary_word_count=[]
for i in data['cleaned_text']:
  text_word_count.append(len(i.split()))
for i in data['cleaned_summary']:
  summary_word_count.append(len(i.split()))
length_df = pd.DataFrame({'text':text_word_count, 'summary':summary_word_count})
length_df.hist(bins = 30)
plt.show()

In [ ]:
count = 0
for i in data['cleaned_summary']:
  if(len(i.split())<=8):
    count += 1
  print(count/len(data['cleaned_summary']))

In [ ]:
max_text_len=30
max_summary_len=8

In [ ]:
cleaned_text = np.array(data['cleaned_text'])
cleaned_summary=np.array(data['cleaned_summary'])
short_text=[]
short_summary=[]
for i in range(len(cleaned_text)):
  if(len(cleaned_summary[i].split())<=max_summary_len and len(cleaned_text[i].split())<=max_test_len):
    short_text.append(cleaned_text[i])
    short_summary.append(cleaned_summary[i])
df=pd.DataFrame({'text':short_text,'summary':short_summary})

In [ ]:
import pandas as pd

# Assuming you have the data in a dictionary as follows
data = {
    'summary': ["Summary 1", "Summary 2", "Summary 3", "Summary 4"]
}

# Create the DataFrame
df = pd.DataFrame(data)

# Apply the lambda function to add 'sostok' and 'eostok' to each value in the 'summary' column
df['summary'] = df['summary'].apply(lambda x: 'sostok' + x + 'eostok')

# Display the updated DataFrame
print(df)


                 summary
0  sostokSummary 1eostok
1  sostokSummary 2eostok
2  sostokSummary 3eostok
3  sostokSummary 4eostok


In [ ]:
import pandas as pd
df['summary'] = df['summary'].apply(lambda x : 'sostok'+ x +'eostok')
df = pd.DataFrame(data)


In [ ]:
import pandas as pd

# Assuming you have the data in a dictionary as follows
data = {
    'text': ["Text 1", "Text 2", "Text 3", "Text 4"]
}

# Create the DataFrame
df = pd.DataFrame(data)

# Apply the lambda function to add 'sostok' and 'eostok' to each value in the 'text' column
df['text'] = df['text'].apply(lambda x: 'sostok' + x + 'eostok')

# Display the updated DataFrame
print(df)


                 text
0  sostokText 1eostok
1  sostokText 2eostok
2  sostokText 3eostok
3  sostokText 4eostok


In [ ]:
from sklearn.model_selection import train_test_split
x_tr,x_val,y_tr,y_val=train_test_split(np.array(df['text']), np.array(df['summary']), test_size=0.1, random_state=0, shuffle=True)

In [ ]:
!pip install --upgrade keras

In [ ]:
!pip install tensorflow


In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

#prepare a tokenizer for reviews on training data
x_tokenizer = Tokenizer()
x_tokenizer.fit_on_texts (list(x_tr))



In [ ]:
thresh=4
cnt=0
tot_cnt=0
freq=0
tot_freq=0

for key, value in x_tokenizer.word_counts.items():
    tot_cnt=tot_cnt+1
    tot_freq=tot_freq+value
    if (value<thresh):
        cnt=cnt+1
        freq=freq+value
print("% of rare words in vocabulary:",(cnt/tot_cnt)*100)
print("Total Coverage of rare words:",(freq/tot_freq)*100)


In [ ]:
x_tokenizer = Tokenizer(num_words=tot_cnt-cnt)
x_tokenizer.fit_on_texts(list(x_tr))
x_tr_seq = x_tokenizer.texts_to_sequences(x_tr)
x_val_seq = x_tokenizer.texts_to_sequences(x_val)
x_tr = pad_sequences(x_tr_seq, maxlen=max_text_len, padding='post')
x_val = pad_sequences(x_val_seq, maxlen=max_text_len, padding='post')
x_voc = x_tokenizer.num_words + 1

In [ ]:
x_voc

1

In [ ]:
y_tokenizer = Tokenizer()
y_tokenizer.fit_on_texts (list(y_tr))

In [ ]:
from keras.preprocessing.text import Tokenizer



In [ ]:
thresh=6
cnt=0
tot_cnt=0
freq=0
tot_freq=0

for key, value in y_tokenizer.word_counts.items():
    tot_cnt=tot_cnt+1
    tot_freq=tot_freq+value
    if (value<thresh):
        cnt=cnt+1
        freq=freq+value
print("% of rare words in vocabulary:",(cnt/tot_cnt)*100)
print("Total Coverage of rare words:",(freq/tot_freq)*100)


In [ ]:
y_tokenizer = Tokenizer(num_words=tot_cnt-cnt)
y_tokenizer.fit_on_texts(list(y_tr))
y_tr_seq = y_tokenizer.texts_to_sequences(y_tr)
y_val_seq = y_tokenizer.texts_to_sequences(y_val)
y_tr = pad_sequences(y_tr_seq, maxlen=max_text_len, padding='post')
y_val = pad_sequences(y_val_seq, maxlen=max_text_len, padding='post')
y_voc = y_tokenizer.num_words + 1

In [ ]:
y_tokenizer.word_counts['sostok'],len(y_tr)

In [ ]:
ind=[]
for i in range(len(y_tr)):
  cnt=0
  for j in y_tr[i]:
    if j!=0:
      cnt=cnt+1
  if(cnt==1):
    ind.append(i)
  y_val=np.delete(y_val,ind, axis=0)
  x_val=np.delete(x_val,ind, axis=0)

In [ ]:
from keras import backend as K
K.clear_session()
latent_dim = 300
embedding_dim=100
encoder_inputs = Input(shape=(max_text_len,))
enc_emb = Embedding(x_voc, embedding_dim, trainable=True)(encoder_inputs)
encoder_lstm1 = LSTM(latent_dim,return_sequences=True,return_state=True,dropout=0.4,recurrent_dropout=0.4)
encoder_output1, state_h1, state_c1 = encoder_lstm1(enc_emb)
encoder_lstm1 = LSTM(latent_dim,return_sequences=True,return_state=True,dropout=0.4,recurrent_dropout=0.4)
encoder_output1, state_h2, state_c2 = encoder_lstm2(encoder_output1)
encoder_lstm1 = LSTM(latent_dim,return_sequences=True,return_state=True,dropout=0.4,recurrent_dropout=0.4)
encoder_output1, state_h3, state_c3 = encoder_lstm1(enc_output2)
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(y_voc, embedding_dim,trainable=True)
dec_emb = dec_emb_layer(decoder_inputs)
decoder_lstm = LSTM(latent_dim,return_sequences=True,return_state=True,dropout=0.4,recurrent_dropout=0.2)
decoder_outputs,decoder_fwd_state,decoder_back_state = decoder_lstm(dec_emb,initial_state=[state_h,state_c])
attn_layer = AttentionLayer(name='attention_layer')
attn_out, attn_states = attn_layer([encoder_outputs,decoder_outputs])
decoder_concat_input = Concatenate(axis=-1,name='concat_layer')([decoder_outputs,attn_out])
decoder_dense = TimeDistributed(Dense(y_voc,activation='softmax'))
decoder_outputs = decoder_dense(decoder_concat_input)
model = Model([encoder_inputs,decoder_inputs], decoder_outputs)
model.summary()


In [ ]:
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=2)


In [ ]:
history=model.fit([x_tr,y_tr[:,:-1]], y_tr.reshape(y_tr.shape[0],y_tr.shape[1],1)[:,1:],epochs=50,callbacks=[es],batch_size=128, validation_data=([x_val,y_val[:,:-1]],y_val.reshape))

In [ ]:
reverse_target_word_index=y_tokenizer.index_word
reverse_source_word_index=x_tokenizer.index_word
target_word_index=y_tokenizer.word_index

In [ ]:

from keras.models import Model



In [ ]:
from tensorflow.keras.models import Model


In [ ]:
encoder_model = Model(inputs=encoder_inputs,outputs=[encoder_outputs, state_h, state_c])
# Below tensors will hold the states of the previous time step

decoder_state_input_h= Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_hidden_state_input = Input (shape=(max_text_len, latent_dim))

#Get the embeddings of the decoder sequence

dec_emb2= dec_emb_layer (decoder_inputs) # To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=[decoder_state_input_h, decoder_state_input_c])

#attention inference

attn_out_inf, attn_states_inf = attn_layer([decoder_hidden_state_input, decoder_outputs2])
decoder_inf_concat = Concatenate (axis=-1, name='concat')([decoder_outputs2, attn_out_inf])

# A dense softmax layer to generate prob dist. over the target vocabulary
decoder_outputs2 = decoder_dense(decoder_inf_concat)

# Final decoder model

decoder_model = Model(
    [decoder_inputs] + [decoder_hidden_state_input, decoder_state_input_h, decoder_state_input_c],
    [decoder_outputs2] + [state_h2, state_c2])


In [ ]:
def decode_sequence(input_seq): #Encode the input as state vectors.
    e_out, e_h, e_c = encoder_model.predict(input_seq)

# Generate empty target sequence of length 1.
    target_seq= np.zeros((1,1))

# Populate the first word of target sequence with the start word.
    target_seq[0, 0] = target_word_index['sostok']

    stop_condition = False

    decoded_sentence = ' '

    while not stop_condition:

        output_tokens, h, c = decoder_model.predict([target_seq] + [e_out, e_h, e_c])

        #Sample a token

        sampled_token_index = np.argmax(output_tokens[e, -1, :])
        sampled_token = reverse_target_word_index[sampled_token_index]

        if(sampled_token!='eostok"):
            decoded_sentence += ' '+sampled_token

        if(sampled_token == 'eostok' or len(decoded_sentence.split()) >= (max_summary_len-1)):
            stop_condition = True

        target_seq = np.zeroes((1,1))

      # Update the target sequence (of length 1).

        target_seq[e, e] = sampled_token_index

      # Update internal states
        e_h, e_c = h c

    return decoded_sentence


In [ ]:
def seq2summary(input_seq):
    newstring=''
    for i in input_seq:
        if((i!=0 and i!=target_word_index['sostok']) and i!=target_word_index['eostok']):
          newstring=newstring+reverse_target_word_index[i]+' '
    return newstring

def seq2text(input_seq):
    newstring=' '
    for i in input_seq:
        if(i!=0):
            newString=newString+reverse_source_word_index[i]+' '

    return newstring



In [ ]:
for i in range(0,100):
    print("Review: ", seq2text(x_tr[i]))
    print("Original summary:", seq2summary(y_tr[i]))
    print("Predicted summary:", decode_sequence (x_tr[i].reshape(1,max_text_len)))
    print("\n")